In [1]:
from utils import call_spotify_db, call_twitter_db
from flask import Flask, jsonify, render_template
from flask import Flask, render_template, redirect, jsonify
import pandas as pd
from sqlalchemy import create_engine, func
import json
import tweepy
import numpy as np
import re
from textblob import TextBlob, Word, Blobber


In [2]:
connection_string = "postgres:12345@localhost:5432/twitter_sentiments"
engine = create_engine(f'postgresql://{connection_string}')
app = Flask(__name__)

In [3]:
spotify_df = pd.read_sql('select * from spotifydb', engine)



In [4]:
file_path = "/Users/svanrooi/Desktop/Humming-Birds/config.json"
with open(file_path) as fp:
    config = json.loads(fp.read())

print(config['KEY'])

1Mm1HogZSqkaHkS9xTEgCXFmL


In [5]:
    # Twitter Api Cred.
key = (config['KEY'])
secret = (config['SECRET'])
bear = (config['BEAR'])
token = (config['ACC_TOKE'])
token_secr = (config['ACC_SECR'])

# Creating the auth object
auth = tweepy.OAuthHandler(key, secret)
# Setting token and access secret
auth.set_access_token(token, token_secr)
# Creating the api call
api = tweepy.API(auth, wait_on_rate_limit=True)

In [6]:
   post = api.user_timeline(screen_name="JoeBiden",
                             count=100, lang="en", tweet_mode="extended")

Unexpected parameter: lang


In [7]:
 df = pd.DataFrame([tweet.full_text for tweet in post], columns=["Tweets"])

In [8]:
    # Creating the auth object
    auth = tweepy.OAuthHandler(key, secret)
    # Setting token and access secret
    auth.set_access_token(token, token_secr)
    # Creating the api call
    api = tweepy.API(auth, wait_on_rate_limit=True)

    # Testing Tweet call
    post = api.user_timeline(screen_name="JoeBiden",
                             count=100, lang="en", tweet_mode="extended")

    # Print 10 tweets
    i = 1
    print("Showing the 10 most recent tweets: \n")
    for tweet in post[0:10]:
        print(str(i) + ")" + tweet.full_text + "\n")
    i = i + 1

    df = pd.DataFrame([tweet.full_text for tweet in post], columns=["Tweets"])

    df.head(11)

    def cleanTxt(text):
        #removing @mentions
        text = re.sub('@[A-Za-z0-9]+', '', text)
    # Removing the "#" symbol
        text = re.sub(r"#", "", text)
    # Removing RT
        text = re.sub(r"RT[\s]+", '', text)
    # Remove the hyper link
        text = re.sub(r"https?:\/\/S+", '', text)
        return text

    # Cleaned tweets down to just text
    df['Tweets'] = df['Tweets'].apply(cleanTxt)

    # Show the cleaned text
   # df=df.dropna()

    # Getting the subjectivity telling how opinionated the tweet is
    def getSubjectivity(text):
        return TextBlob(text).sentiment.subjectivity

    # Get polarity to tell how positive or negative tweet is
    def getPolarity(text):
        return TextBlob(text).sentiment.polarity

    # Adding columns for subjectivity and polarity
    df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)

    df['Polarity'] = df['Tweets'].apply(getPolarity)

    def getAnalysis(score):
        if score < 0:
            return 'Negative'
        elif score == 0:
            return 'Neutral'
        else:
            return 'Positive'

    df['Sentiment'] = df['Polarity'].apply(getAnalysis)

    np.random.choice([["Positive", "Negative", "Neutral"], (10, 1)])

    conditions = [
        (df["Sentiment"] == "Positive"),
        (df["Sentiment"] == "Negative"),
        (df["Sentiment"] == "Neutral")
    ]

    values = ['0', '1', '2']

    df["Sentiment_Num"] = np.select(conditions, values)

Unexpected parameter: lang


Showing the 10 most recent tweets: 

1)In February, the economy created 678,000 jobs, following the greatest year of job growth in American history and the fastest economic growth in almost 40 years.

1)My plan to fight inflation will lower your everyday costs and reduce the deficit. 
https://t.co/3fudH3s4Ck

1)Too many families are still struggling with higher costs. I get it. Our top priority must be getting prices under control. That’s why we’re going to create more jobs that you can earn a living, a good living, right here in America, and invest in infrastructure and innovation.

1)Since I took office, the economy has created 7.4 million jobs. That’s 7.4 million jobs providing families with dignity and a little more breathing room. 

We are building a better America.

1)RT @POTUS: Our economy has now added 7.4 million jobs since I took office. 678,000 jobs added just last month. And unemployment is down to…

1)To the Russian oligarchs and corrupt leaders who have bilked billions of

/var/folders/d0/k96gp7tn0ql30t5ybmy3sv7h0000gp/T/ipykernel_5312/3635352121.py:63: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.random.choice([["Positive", "Negative", "Neutral"], (10, 1)])


In [9]:
df.Sentiment.value_counts()

Sentiment = df.Sentiment.value_counts().to_dict()
Sentiment = max(Sentiment, key=Sentiment.get)

print("Your Tweets look really "+Sentiment+"!")


Your Tweets look really Positive!


In [10]:
spotify_df

,artist,track,genre,popularity,sentiment,sentiment_score
0,Glass Animals,Heat Waves,gauze pop,95,Positive,81.28
1,Glass Animals,Heat Waves,indietronica,95,Positive,81.28
2,Glass Animals,Heat Waves,shiver pop,95,Positive,81.28
3,The Kid LAROI,STAY (with Justin Bieber),australian hip hop,93,Negative,92.17
4,Imagine Dragons,Enemy (with JID) - from the series Arcane Leag...,modern rock,85,Negative,93.25
...,...,...,...,...,...,...
132,Sebastian Yatra,Tacones Rojos,colombian pop,80,Positive,45.88
133,Sebastian Yatra,Tacones Rojos,latin,80,Positive,45.88
134,Sebastian Yatra,Tacones Rojos,reggaeton colombiano,80,Positive,45.88
135,Blessd,Medallo,reggaeton,92,Negative,79.06


In [11]:
print(spotify_df["sentiment"][0])

Positive


In [12]:
spotify_df.head()

,artist,track,genre,popularity,sentiment,sentiment_score
0,Glass Animals,Heat Waves,gauze pop,95,Positive,81.28
1,Glass Animals,Heat Waves,indietronica,95,Positive,81.28
2,Glass Animals,Heat Waves,shiver pop,95,Positive,81.28
3,The Kid LAROI,STAY (with Justin Bieber),australian hip hop,93,Negative,92.17
4,Imagine Dragons,Enemy (with JID) - from the series Arcane Leag...,modern rock,85,Negative,93.25


In [16]:
data = spotify_df[(spotify_df["sentiment"]== Sentiment)].T.to_dict()


In [17]:
data

{0: {'artist': 'Glass Animals',
  'track': 'Heat Waves',
  'genre': 'gauze pop',
  'popularity': 95,
  'sentiment': 'Positive',
  'sentiment_score': 81.28},
 1: {'artist': 'Glass Animals',
  'track': 'Heat Waves',
  'genre': 'indietronica',
  'popularity': 95,
  'sentiment': 'Positive',
  'sentiment_score': 81.28},
 2: {'artist': 'Glass Animals',
  'track': 'Heat Waves',
  'genre': 'shiver pop',
  'popularity': 95,
  'sentiment': 'Positive',
  'sentiment_score': 81.28},
 15: {'artist': 'Elton John',
  'track': 'Cold Heart - PNAU Remix',
  'genre': 'glam rock',
  'popularity': 78,
  'sentiment': 'Positive',
  'sentiment_score': 69.08},
 16: {'artist': 'Elton John',
  'track': 'Cold Heart - PNAU Remix',
  'genre': 'mellow gold',
  'popularity': 78,
  'sentiment': 'Positive',
  'sentiment_score': 69.08},
 17: {'artist': 'Elton John',
  'track': 'Cold Heart - PNAU Remix',
  'genre': 'piano rock',
  'popularity': 78,
  'sentiment': 'Positive',
  'sentiment_score': 69.08},
 18: {'artist': 'L